### Abstract

In imputing missing precipitation data using machine learning (ML) techniques, it is crucial to determine **how many** stations, and **which nearby stations**, can be used as features for imputation. This code addresses this question by evaluating station proximity, where stations are grouped based on Euclidean distances calculated from their geographic coordinates. These stations are then analyzed in 10 km intervals. Missing values are imputed using the **HistGradientBoostingRegressor model**, with the performance of the imputation assessed through multiple metrics, including R², RMSE, MAE, and cross-validation scores. The results highlight the relationship between spatial proximity and the imputation accuracy, identifying optimal station groupings for reliable predictions. This approach offers valuable insights into the impact of station proximity on ML-based imputation for precipitation data.

### Expected Results

The goal of this analysis is to identify the most suitable group of nearby stations for imputing missing precipitation data for each target station. The results will provide insight into how spatial proximity impacts the accuracy of imputation using the HistGradientBoostingRegressor model. Based on the evaluation of R² scores and mean R² from cross-validation, we expect the following outcomes:

1. **Identification of Optimal Distance Group**: 
   For each target station, the evaluation of different distance groups (e.g., 0-10 km, 0-20 km, 0-30 km, etc.) will reveal which group of nearby stations yields the best imputation performance. The group with the highest R² score and an acceptable cross-validation mean R² score will be considered the most suitable for imputing missing values.

2. **Performance Metrics**:
   - **R² Score**: We expect the R² score to vary across groups and stations, with higher values indicating better imputation performance. For example, we may observe R² scores ranging from 0.7 to 0.8 or higher for the most suitable groups.
   - **Mean R² from Cross-Validation**: Cross-validation will help ensure the model's robustness. We expect to find that groups with the highest R² also have a relatively high mean R² from cross-validation, indicating stable and reliable imputation performance across different subsets of the data.

3. **Distance Consideration**:
   As the distance between stations increases, the R² score may decrease due to weaker correlations between distant stations. However, we expect that certain target stations will perform well even with stations located further away, especially if those stations exhibit similar climatic or environmental conditions. For example, a group 0-70 km or 0-100 km might provide an optimal balance of spatial proximity and prediction accuracy.

4. **Group Recommendations**:
   Based on the results, each target station will have a recommended distance group for imputation, providing insight into the relationship between station proximity and imputation accuracy. For example, a station like "19-936" may have the best performance with the "0-80 km" group, while another station, such as "19-937," may perform better with the "0-100 km" group.

5. **Suitability of Nearby Stations**:
   The results will also help identify the number of nearby stations suitable for imputing missing values for each target station. The evaluation will highlight whether stations close to the target station are more appropriate, or if using stations that are further away still provides acceptable performance.

Overall, this analysis will offer valuable insights into the importance of station proximity for accurate precipitation data imputation and guide the selection of suitable stations for future imputation tasks.


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from scipy.spatial.distance import cdist

# Load your dataset
file_path = 'F:/precip/precip/paper/final_filtered_data33.xlsx'
df = pd.read_excel(file_path)

# Define the date range of interest
start_date = 136801
end_date = 139612

# Convert 'Date' to numeric if it's not already
df['Date'] = pd.to_numeric(df['Date'], errors='coerce')

# Pivot the DataFrame to get stations as columns and Date as index
pivot_df = df.pivot(index='Date', columns='station', values='precipitation')

# Filter the data by the specified date range
filtered_df = pivot_df[(pivot_df.index >= start_date) & (pivot_df.index <= end_date)]

# Extract location coordinates
locations = df[['station', 'X', 'Y']].drop_duplicates().set_index('station')

# Calculate Euclidean distances between stations
def calculate_distances(locations):
    coords = locations[['X', 'Y']].values
    distances = cdist(coords, coords, metric='euclidean')
    return distances

# Convert distances to kilometers (assuming UTM coordinates in meters)
def convert_to_kilometers(distances):
    return distances / 1000

# Calculate and convert distances
distances = calculate_distances(locations)
distances_km = convert_to_kilometers(distances)

# Identify stations with missing values
stations_with_missing_data = filtered_df.columns[filtered_df.isnull().any()]

# Dynamically create distance groups, increasing by 10 km intervals
distance_step = 10
max_distance = np.max(distances_km)  # Use max distance between all stations
distance_groups = [(0, i) for i in range(distance_step, int(np.ceil(max_distance)) + distance_step, distance_step)]

# Dictionary to store evaluation metrics for each station and group
evaluation_results = []

# Function to find suitable stations based only on distance within a group
def find_suitable_stations_by_group(station_with_missing_data, group):
    suitable_stations = []
    
    for station in filtered_df.columns:
        if station != station_with_missing_data:
            # Calculate distance between the current station and other stations
            distance = distances_km[locations.index.get_loc(station_with_missing_data), locations.index.get_loc(station)]
            if group[0] <= distance < group[1]:
                suitable_stations.append(station)
    
    return suitable_stations

# Evaluate model performance for each station with missing values
for target_station in stations_with_missing_data:
    print(f"\nEvaluating for target station: {target_station}")
    
    for group in distance_groups:
        group_name = f"{group[0]}-{group[1]} km"
        print(f"\nEvaluating group: {group_name}")
        
        # Find suitable stations for the target station
        suitable_stations = find_suitable_stations_by_group(target_station, group)
        
        if suitable_stations:
            combined_df = filtered_df[[target_station] + suitable_stations].copy()
            combined_df = combined_df.dropna(subset=[target_station])
            
            if len(combined_df) < 10:  # Example check for minimum data points
                print(f"Not enough data points for station {target_station} in group {group_name}.")
                continue
            
            X = combined_df[suitable_stations].values
            y = combined_df[target_station].values
            
            valid_indices = ~np.isnan(y)
            X = X[valid_indices]
            y = y[valid_indices]
            
            # Define the pipeline
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('regr', HistGradientBoostingRegressor())
            ])
            
            # Perform cross-validation and calculate average metrics
            cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')  # Using R² as the scoring metric
            mean_r2 = np.mean(cv_scores)
            
            # Calculate additional metrics using a split for demonstration
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            pipeline.fit(X_train, y_train)
            y_pred = pipeline.predict(X_test)
            
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            evaluation_results.append({
                'Distance Group': group_name,
                'Station': target_station,
                'Mean R2 CV': mean_r2,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae,
                'R2': r2
            })
            
            print(f"R² Score for station {target_station} in group {group_name}: {r2:.4f}")
            print(f"Mean R² from Cross-Validation for station {target_station} in group {group_name}: {mean_r2:.4f}")
        else:
            print(f"No suitable stations found for station {target_station} in group {group_name}.")

# Convert results to DataFrames
results_df = pd.DataFrame(evaluation_results)

# Optionally, save the evaluation results to an Excel file
results_df.to_excel('F:/precip/precip/paper/evaluation_results_consolidated.xlsx', index=False)



Evaluating for target station: 19-936

Evaluating group: 0-10 km
No suitable stations found for station 19-936 in group 0-10 km.

Evaluating group: 0-20 km
R² Score for station 19-936 in group 0-20 km: 0.7963
Mean R² from Cross-Validation for station 19-936 in group 0-20 km: 0.7490

Evaluating group: 0-30 km
R² Score for station 19-936 in group 0-30 km: 0.7527
Mean R² from Cross-Validation for station 19-936 in group 0-30 km: 0.7318

Evaluating group: 0-40 km
R² Score for station 19-936 in group 0-40 km: 0.7917
Mean R² from Cross-Validation for station 19-936 in group 0-40 km: 0.7135

Evaluating group: 0-50 km
R² Score for station 19-936 in group 0-50 km: 0.7918
Mean R² from Cross-Validation for station 19-936 in group 0-50 km: 0.7007

Evaluating group: 0-60 km
R² Score for station 19-936 in group 0-60 km: 0.7947
Mean R² from Cross-Validation for station 19-936 in group 0-60 km: 0.7032

Evaluating group: 0-70 km
R² Score for station 19-936 in group 0-70 km: 0.8198
Mean R² from Cross-V

D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


R² Score for station 31-101 in group 0-120 km: 0.7400
Mean R² from Cross-Validation for station 31-101 in group 0-120 km: 0.7281

Evaluating group: 0-130 km


D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


R² Score for station 31-101 in group 0-130 km: 0.7540
Mean R² from Cross-Validation for station 31-101 in group 0-130 km: 0.7274

Evaluating group: 0-140 km


...

R² Score for station 31-101 in group 0-300 km: 0.7455
Mean R² from Cross-Validation for station 31-101 in group 0-300 km: 0.7344

Evaluating group: 0-310 km


D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


R² Score for station 31-101 in group 0-310 km: 0.7455
Mean R² from Cross-Validation for station 31-101 in group 0-310 km: 0.7344

Evaluating group: 0-320 km


D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
D:\anacon\envs\sep_2021\Lib\site-packages\sklearn\utils\extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


R² Score for station 31-101 in group 0-320 km: 0.7455
Mean R² from Cross-Validation for station 31-101 in group 0-320 km: 0.7344

Evaluating for target station: 31-109

Evaluating group: 0-10 km
No suitable stations found for station 31-109 in group 0-10 km.

Evaluating group: 0-20 km
R² Score for station 31-109 in group 0-20 km: 0.3972
Mean R² from Cross-Validation for station 31-109 in group 0-20 km: 0.4435

Evaluating group: 0-30 km
R² Score for station 31-109 in group 0-30 km: 0.4883
Mean R² from Cross-Validation for station 31-109 in group 0-30 km: 0.5340

Evaluating group: 0-40 km
R² Score for station 31-109 in group 0-40 km: 0.6223
Mean R² from Cross-Validation for station 31-109 in group 0-40 km: 0.5657

Evaluating group: 0-50 km
R² Score for station 31-109 in group 0-50 km: 0.6623
Mean R² from Cross-Validation for station 31-109 in group 0-50 km: 0.6178

Evaluating group: 0-60 km
R² Score for station 31-109 in group 0-60 km: 0.6280
Mean R² from Cross-Validation for station 31-